In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


In [73]:
data = pd.read_csv('../data/datadaily/data_daily0.csv', index_col='Date_time_nr')
data_ml = feature_pipeline(data)


In [74]:
data_ml.head(1)

,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,wind_sin,wind_cos,year_sin,year_cos,P_avg
Date_time_nr,,,,,,,,,,,
1356998400,5.314167,1012.458333,88.083333,5.037083,0.332917,0.0,-0.5373,-0.843391,0.017202,0.999852,782.613541


In [ ]:
from keras import Model, Input
from keras.layers  import Dense, Flatten, Concatenate, Normalization, Cropping1D, Lambda

def create_model(input_shape1 = (6, ), input_shape2 = (4, )):
    inputs1 = Input(input_shape1)
    inputs2 = Input(input_shape2)
    x = Normalization(inputs1)
    z = Concatenate()([x, inputs2])
    y = Dense(8)(z)
    
    model = Model(inputs=[inputs1, inputs2], outputs=y)

    opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss="mse"
        , optimizer=opt
        , metrics=[keras.metrics.RootMeanSquaredError()]
        )
    return model

#model = create_model()

In [ ]:
def create_model(input_shape = (10, )):
    inputs = Input(input_shape)
    x1 = Cropping1D((0,4))
    x2 =  Cropping1D((6,0))
    x1 = Normalization(x1)
    z = Concatenate()([x1, x2])
    y = Dense(8)(z)
    
    model = Model(inputs=[inputs1, inputs2], outputs=y)

    opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss="mse"
        , optimizer=opt
        , metrics=[keras.metrics.RootMeanSquaredError()]
        )
        
    return model

# utils

In [71]:
from sklearn.preprocessing import MinMaxScaler 
def add_yearly_fourier(df): 
    ts = pd.to_datetime(df.index) 
    day_of_year = ts.dayofyear 
    T = 365.25 
    df = df.copy() 
    df["year_sin"] = np.sin(2 * np.pi * day_of_year / T) 
    df["year_cos"] = np.cos(2 * np.pi * day_of_year / T) 
    return df 

def encode_wind_direction_circular(df): 
    df = df.copy() 
    theta = np.deg2rad(df["wind_deg"] % 360) 
    df["wind_sin"] = np.sin(theta) 
    df["wind_cos"] = np.cos(theta)
    df.drop(columns=["wind_deg"], inplace=True) 
    return df 

def feature_pipeline(df):
    if 'P_avg' in df.columns:
        df_X = df.drop(columns=['P_avg'])
        df_y = df['P_avg']
        df_X = encode_wind_direction_circular(df_X) 
        df_X = add_yearly_fourier(df_X) 
        df = pd.concat([df_X, df_y], axis=1)
        return df
    
    else:
        df = encode_wind_direction_circular(df) 
        df = add_yearly_fourier(df) 
        return df